# FleetTrack Tech Residency 40

In [2]:
import numpy as np
import pandas as pd
import time
import json
import random

### 4 hours of data for one vehicle

In [11]:
#initiate random seed for reproducibility
np.random.seed(42)

speeds = [30] # Start around 30 mph and add small random changes
odometer = [87763.24]  # starting mileage
fuel = [20.36]  # starting fuel level in gallons
engine_temp = [190]  # starting engine temperature in Fahrenheit
mpg = 18  # miles per gallon
lat = [33.45]
long = [-112.07]

for _ in range(1, 24):
    change = np.random.uniform(-5, 5)
    lat.append(lat[-1] + random.uniform(-0.05, 0.05))
    long.append(long[-1] + random.uniform(-0.05, 0.05)),
    #speed, odometer, and fuel consumption are related calculations
    new_speed = max(0, min(70, speeds[-1] + change))
    speeds.append(new_speed)
    odometer.append(odometer[-1] + new_speed * (10/60))
    fuel_consumption = new_speed * (10/60) / mpg
    fuel.append(max(0, fuel[-1] - fuel_consumption))
    #engine temperature not related to any other variable
    engine_temp.append(min(250, engine_temp[-1] + np.random.uniform(0, 5)))

truck2_20231005 = pd.DataFrame({
    "vehicle_id": ["truck2"] * 24,
    "latitude": np.round(lat, 5),
    "longitude": np.round(long, 5),
    "speed_mph": np.round(speeds, 2),
    "check_engine": [0] * 24,
    "odometer": np.round(odometer, 2),
    "fuel_gal": np.round(fuel, 2),
    "engine_temp_f": np.round(engine_temp, 2),
    #stagger time with car data in case we need to merge later
    "recorded_at": pd.date_range(start='2023-10-05 09:02:00', periods=24, freq='10min')
})

truck2_20231005.to_json("truck2_20231005.json", orient="records", indent=2)

truck2_20231005.tail()

,vehicle_id,latitude,longitude,speed_mph,check_engine,odometer,fuel_gal,engine_temp_f,recorded_at
19,truck2,33.55750,-112.00483,12.88,0,87823.08,17.04,236.69,2023-10-05 12:12:00
20,truck2,33.51659,-111.95790,14.72,0,87825.54,16.90,238.89,2023-10-05 12:22:00
21,truck2,33.50995,-111.94625,10.94,0,87827.36,16.80,241.37,2023-10-05 12:32:00
22,truck2,33.48561,-111.95629,6.28,0,87828.41,16.74,245.92,2023-10-05 12:42:00
23,truck2,33.44478,-111.96826,3.87,0,87829.05,16.70,249.23,2023-10-05 12:52:00
